# Lab Automation Deck Calibration
After installing a lab automation deck on the machine, we need to record reference positions for each of the six slots for exact alingment. Step through this notebook to create a lab_automation_deck_config.json file!

This notebook assumes you have a top-down camera tool setup on your machine. If you don't, you can use another tool to manually align each offset.

This notebook also uses

In [ ]:
from science_jubilee.Machine import Machine, get_root_dir
from science_jubilee.tools.Camera import Camera
from jinja2 import Environment, FileSystemLoader, select_autoescape
import json
import os
%load_ext autoreload
%autoreload 2

In [ ]:
# Set up the calibration file
# The following default values apply to the standard lab automation deck
# Adapt them if you have customized you deck!
deck_type = "Lab Automation Deck" # What type of deck is this?
num_slots = 6 # How many slots are there
num_sharps_containers = 0 # How many sharps containers are you using, if any?
slot_type = "SLAS Standard Labware" # What do these slots hold?
plate_material = "Aluminum" # What is your Jubilee bed plate material
mask_material = "Delrin" # What material is your deck made of?

# Your lab automation deck slots will have 1 corner with no flexure element
# Specify whether this is the top_left, top_right, bottom_left, or bottom_right
# where 'right' means larger x values and 'top' means larger y values
offset_corner = "bottom_left" # What corner are you offsetting from?

In [ ]:
# We'll populate slot_data using this set_slot_data function
slot_data = {} 
def set_slot_data(slot_index: int):
    position = m.get_position()
    slot_offset = [float(position['X']), float(position['Y'])]
    slot_data[slot_index] = slot_offset

In [ ]:
# Initialize your machine connection
m = Machine()

In [ ]:
# Check which tools are currently configured on your machine
m._configured_tools

In [ ]:
# Load your camera tool
# Change this to match the index of your camera tool
camera = Camera(index=3, name="top_down_camera") 
m.load_tool(camera)

In [ ]:
m.pickup_tool(camera)

In [ ]:
# Move to your camera's focus height
# For me, that's z=30mm
m.move_to(z=30)

In [ ]:
# The 0th slot is closest to the machine's (0,0)
# Open a camera feed and position the camera using the duet web controller
# Move the camera so that the center of the camera is over the slot corner specified above as 'offset_corner'
# press esc when done to close the camera feed
m.move_to(x=30, y=30)
camera.video_stream()

In [ ]:
# Save this position
slot_index = 0
set_slot_data(slot_index)

In [ ]:
# Now, repeat this in the following cells for each of the other slots!
# Be sure tolign to the same corner for each slot

In [ ]:
# Slot 1
m.move(dx=140) # Move to approximate position of slot 1
camera.video_stream() # Fine tune the position using the camera feed

In [ ]:
# Save this position
slot_index = 1
set_slot_data(slot_index)

In [ ]:
# Slot 2
m.move(dx=-140, dy=100) # Move to approximate position of slot 2
camera.video_stream() # Fine tune the position using the camera feed

In [ ]:
# Save this position
slot_index = 2
set_slot_data(slot_index)

In [ ]:
# Slot 3
m.move(dx=140) # Move to approximate position of slot 3
camera.video_stream() # Fine tune the position using the camera feed

In [ ]:
# Save this position
slot_index = 3
set_slot_data(slot_index)

In [ ]:
# Slot 4
m.move(dx=-140, dy=100) # Move to approximate position of slot 4
camera.video_stream() # Fine tune the position using the camera feed

In [ ]:
# Save this position
slot_index = 4
set_slot_data(slot_index)

In [ ]:
# Slot 5
m.move(dx=140) # Move to approximate position of slot 5
camera.video_stream() # Fine tune the position using the camera feed

In [ ]:
# Save this position
slot_index = 5
set_slot_data(slot_index)

In [ ]:
# If you have a sharps container installed, manually move to it
# Skip to "Save Calibration File" below if you aren't installing a sharps container
camera.video_stream() 

In [ ]:
# We use negative slot indices for sharps containers
slot_index = -1
set_slot_data(slot_index)

In [ ]:
# Save Calibration File
file_name = "lab_automation_deck_MA" # Change this if you'd like to refer to this calibration by a different name

In [ ]:
# Run this cell to save your calibration file!
deck_config_path = os.path.join(get_root_dir(), "science_jubilee", "decks", "deck_definition", f"{file_name}.json")
env = Environment(loader=FileSystemLoader("templates"))
template = env.get_template("lab_automation_deck_template.json")
calibration_contents = template.render(deck_type=deck_type, num_slots=num_slots, num_sharps_containers=num_sharps_containers, slot_type=slot_type, plate_material=plate_material, mask_material=mask_material, offset_corner=offset_corner, slot_data=slot_data)

with open(deck_config_path, 'w') as f:
    f.write(calibration_contents)

In [ ]:
# Done!